In [94]:
import numpy as np 
import pandas as pd
import portion as P 
import string
import itertools
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable


# Finding reductions for linear problems
Consider the following locally verifiable problem in (3,3)-biregular trees:
- $\Sigma \subseteq [0,1]$
- Task is to label the edges
- Sum of edge labels incident to
    - white nodes is $\geq \alpha$
    - black nodes is $\leq \beta$

This program should find out if any label in some interval $(A_0, A_1)$ can be replaced by a single label.

The basic idea is to check every combination of these intervals and which of these combinations could be white and/or black neighbourhoods. Using these combinations we can form a set of linear inequalities that the simultanious reductions must satisfy. Lastly we use linear programming to search for reductions.


In [95]:
# Define some variables

# Degrees of black and white nodes
d = 3
delta = 3

# The treshold for black and white sums
beta = 2
alpha = 1

# The set of possible labels
Sigma_string = "[0, 1/3) U (2/3, 4/5) U (4/5, 1]"


In [96]:
# Define parser for Sigma
def convert(s):
    try:
        return float(s)
    except ValueError:
        num, denom = s.split('/')
        return float(num) / float(denom)

params = {
    'disj': ' U '
}

Sigma = P.from_string(Sigma_string, conv=convert, **params)

In [97]:
interval_count = len([x for x in Sigma])
intervals = pd.DataFrame({"interval": [x for x in Sigma], "reduction": [None for x in Sigma]}, index=list(string.ascii_uppercase[0:interval_count]))
intervals

,interval,reduction
A,"[0.0,0.25)",None
B,"(0.75,0.8)",None
C,"(0.8,1.0]",None


In [98]:
def sum_intervals(interval_list):
    min = 0
    max = 0
    min_in = True
    max_in = True
    for interval in interval_list:
        min_in = min_in and interval.left == P.CLOSED
        max_in = max_in and interval.right == P.CLOSED
        min += interval.lower
        max += interval.upper
    if min_in and max_in:
        return P.closed(min, max)
    if min_in and not max_in:
        return P.closedopen(min, max)
    if not min_in and max_in:
        return P.openclosed(min, max)
    return P.open(min, max)


In [99]:
# If white and black nodes have same amount of neighbours, some neighborhoods can be suitable for both
if d==delta:
    combinations = list(itertools.combinations_with_replacement(intervals.index, d))
    neighbourhoods = pd.DataFrame({"combination": combinations})
    neighbourhoods["W"] = None 
    neighbourhoods["B"] = None

else:
    combinations = list(itertools.combinations_with_replacement(intervals.index, delta))
    combinations.extend(list(itertools.combinations_with_replacement(intervals.index, d)))
    neighbourhoods = pd.DataFrame({"combination": combinations})
    neighbourhoods["OK"] = None
    


In [100]:
white_range = P.open(alpha, P.inf)
black_range = P.open(-P.inf, beta)

for index, row in neighbourhoods.iterrows():
    range = sum_intervals(list(map(lambda x: intervals.loc[x, "interval"], row["combination"])))
    
    # See if we have to check both white and black ranges or only one
    if d==delta:
        neighbourhoods.at[index, "W"] = not (white_range & range).empty
        neighbourhoods.at[index, "B"] = not (black_range & range).empty
        
    else: 
        if len(row["combination"]) == d:
            print()
            neighbourhoods.at[index, "OK"] = not (black_range & range).empty
            
        else: neighbourhoods.at[index, "OK"] = not (white_range & range).empty
        
    

In [101]:
neighbourhoods

,combination,OK
0,"(A, A, A)",False
1,"(A, A, B)",True
2,"(A, A, C)",True
3,"(A, B, B)",True
4,"(A, B, C)",True
5,"(A, C, C)",True
6,"(B, B, B)",True
7,"(B, B, C)",True
8,"(B, C, C)",True
9,"(C, C, C)",True


In [102]:
def to_retor(neighbourhoods):
    if d == delta:
        white_retor = ""
        black_retor = ""
        for index, row in neighbourhoods.iterrows():
            if row["W"]:
                white_retor += " ".join(row["combination"])
                white_retor += "\n"
            if row["B"]:
                black_retor += " ".join(row["combination"])
                black_retor += "\n"

    else:
        white_retor = ""
        black_retor = ""
        for index, row in neighbourhoods.iterrows():
            if len(row["combination"]) == d and row["OK"]:
                black_retor += " ".join(row["combination"])
                black_retor += "\n"
            elif row["OK"]:
                white_retor += " ".join(row["combination"])
                white_retor += "\n"
    
    return (black_retor, white_retor)

In [103]:
def find_reductions(neighbourhoods):
    model = LpProblem(name="Reductions", sense=LpMaximize)

    # Linear ineq solvers can't handle true inequalities, so maybe it's best to just use epsilon differences?
    epsilon = 0.01

    variables = dict(zip(list(string.ascii_lowercase[0:interval_count]), 
                    [LpVariable(name = symbol, lowBound=0, upBound=1) for symbol in list(string.ascii_lowercase[0:interval_count])]))
    
    # Add the constraints of the original sets
    for index, row in intervals.iterrows():
        variable = variables[index.lower()]
        interval = row["interval"]
        if interval.left == P.CLOSED:
            model += (variable>=interval.lower, f"Init_{index}_lower")
        else:
            model += (variable>=interval.lower + epsilon, f"Init_{index}_lower")
            
        if interval.right == P.CLOSED:
            model += (variable<=interval.upper, f"Init_{index}_upper")
        else:
            model += (variable<=interval.upper - epsilon, f"Init_{index}_upper")


    # Add the constraints imposed by summation of the original sets
    if d == delta:
        for index, row in neighbourhoods.iterrows():
            if (row["W"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) >= alpha, f"Constraint_{index}_W")
            if (row["B"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) <= beta, f"Constraint_{index}_B")

    else:  
        for index, row in neighbourhoods.iterrows():
            if (len(row["combination"])==d) and row["OK"]:
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) <= beta, f"Constraint_{index}_B")
            elif (row["OK"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) >= alpha, f"Constraint_{index}_W")

    if model.solve() == -1:
        print("No reductions found.")
    
    else:
        print("Reductions found: ")
        for var in model.variables()[1:]:
            intervals.loc[var.name.upper(), "reduction"] = var.value()
        
        print(intervals)
        print("\nRound eliminator syntax:")
        black, white = to_retor(neighbourhoods)
        
        print("\n"+black)
        print("\n"+white)


In [104]:
# First check for 0-round solution

easy_solution_interval = P.closed(alpha/delta, beta/d)
easy_solutions = (easy_solution_interval & Sigma)
if not easy_solutions.empty:
    print("0-round solution found.")
    print(f"Choose any single value from {P.to_string(easy_solutions, **params)}.")

# Try to find some reductions
else:
    print("No 0-round solutions found.")
    find_reductions(neighbourhoods)
    
    


No 0-round solutions found.
Reductions found: 
     interval reduction
A  [0.0,0.25)     0.105
B  (0.75,0.8)      0.79
C   (0.8,1.0]      0.81

Round eliminator syntax:

A A
A B
A C


A A B
A A C
A B B
A B C
A C C
B B B
B B C
B C C
C C C

